In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.data.utils import read_embeddings_dataframe

In [4]:
df = read_embeddings_dataframe("embeddings.csv")

In [10]:
type(df['embedding'].iloc[0])

numpy.ndarray

In [5]:
from src.models.clustering import perform_clusterings, create_clustering_files

In [6]:
results = perform_clusterings(df, 5, 5)

In [23]:
results

(           username  kmeans_cluster  gmm_cluster  db_scan_cluster
 0         achybicka               4            3                0
 1          ac_sobol               2            1               -1
 2        adambielan               2            1               -1
 3        adamgaweda               4            3               -1
 4    adamowiczpawel               2            1                0
 ..              ...             ...          ...              ...
 543    zofiaczernow               4            3                0
 544   z_frankiewicz               4            3                0
 545     _mkonieczny               3            1                0
 546      _piotrcwik               3            3                0
 547      _urbaniakj               3            1                0
 
 [548 rows x 4 columns],
 {'k_means': KMeans(n_clusters=5),
  'gmm': GaussianMixture(n_components=5),
  'db_scan': DBSCAN()})

In [7]:
create_clustering_files(results, ".", "clusters", "cluster_models")

In [4]:
from src.models.dimensionality_reduction import create_umap_data_and_model
create_umap_data_and_model(df, ".", "graph_umap", "umap")